In [21]:
library(readr)
library(fgsea)
library(readr)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
all_genes_ranked <- read_csv("/home/miltondp/projects/labs/greenelab/phenoplier/base/data/crispr_screen/lipid_DEG.csv")


── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  gene_name = col_character(),
  GFPLow_vs_UnSorted.log2FC = col_double(),
  GFPLow_vs_UnSorted.FDR = col_double(),
  GFPLow_vs_UnSorted.DEG = col_logical(),
  GFPHigh_vs_UnSorted.log2FC = col_double(),
  GFPHigh_vs_UnSorted.FDR = col_double(),
  GFPHigh_vs_UnSorted.DEG = col_logical(),
  GFPHigh_vs_GFPLow.log2FC = col_double(),
  GFPHigh_vs_GFPLow.FDR = col_double(),
  GFPHigh_vs_GFPLow.DEG = col_logical(),
  `lipid effect` = col_character(),
  rank = col_double()
)




In [3]:
orig_deg_gene_sets <- list()

for (r in unique(all_genes_ranked$rank)) {
    if (r == 0) {
        next
    }
    
    data <- all_genes_ranked[all_genes_ranked$rank == r,]
    #q <- quantile(data, 0.50, names=FALSE)
    
    orig_deg_gene_sets[[paste0("gene_set_", r)]] <- data$gene_name
}

In [4]:
deg_gene_sets <- list()

In [5]:
# genes that increase lipids
deg_gene_sets[["gene_set_increase_2_and_3"]] <- c(
    orig_deg_gene_sets[["gene_set_2"]],
    orig_deg_gene_sets[["gene_set_3"]]
)

In [6]:
# genes that decrease lipids
deg_gene_sets[["gene_set_decrease_-2_and_-3"]] <- c(
    orig_deg_gene_sets[["gene_set_-2"]],
    orig_deg_gene_sets[["gene_set_-3"]]
)

In [7]:
# MultiPLIER LVs
multiplier_z = readRDS("/media/miltondp/Elements1/projects/multiplier/recount2_PLIER_data/recount_PLIER_model.RDS")$Z

lvs = list()
for (cidx in 1:ncol(multiplier_z)) {
    data <- multiplier_z[, cidx]
    # q <- quantile(data, 0.75, names=FALSE)
    q <- 0.0
    
    lvs[[paste0("LV", cidx)]] <- data[data > q]
}

# Compute enrichment on all LVs

In [8]:
n_reps = 10

In [9]:
set.seed(0)

In [10]:
results = list()

for (lv in names(lvs)) {
    repetitions = list()
    
    for (i in 1:n_reps) {
        rep_res <- fgsea(pathways = deg_gene_sets, stats = lvs[[lv]], scoreType = "pos", eps = 0.0)[order(pval), ]
        rep_res[, "leadingEdge"] <- sapply(rep_res$leadingEdge, paste, collapse=",")
        rep_res[, "lv"] <- lv
        rep_res[, "rep_idx"] <- i
        
        repetitions[[i]] <- rep_res
    }
#     res <- fgsea(pathways = deg_gene_sets, stats = lvs[[lv]], scoreType = "pos", eps = 0.0)[order(pval), ]
    res <- do.call(rbind, repetitions)
#     res[, "leadingEdge"] <- sapply(res$leadingEdge, paste, collapse=",")
#     res[, "lv"] <- lv
    results[[lv]] <- res
}

In [11]:
length(results)

[1] 987

In [12]:
df <- do.call(rbind, results)

In [13]:
dim(df)

[1] 19730    10

In [14]:
head(df)

pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
gene_set_decrease_-2_and_-3,0.1848152,0.2357642,0.09592068,0.5784179,1.126666,35,"PTBP1,KEAP1,PEX14,DLST,PCYT2,MAD2L2,GLRX5,OGDH,UBE2J2,CSK",LV1,1
gene_set_increase_2_and_3,0.2357642,0.2357642,0.08220549,0.5391549,1.076511,63,"CHERP,RANGAP1,HNRNPL,RPS2,E4F1,TAF1C,GATAD2A,SAFB,TAF6,FBL,LSM4,SUPT5H,CHD4,PFDN6,SLC35B2,USP39,POLR3E,POLR2C,RPLP0,SREBF2,ACACA,RPL18",LV1,1
gene_set_decrease_-2_and_-3,0.1988012,0.2517483,0.09167952,0.5784179,1.123767,35,"PTBP1,KEAP1,PEX14,DLST,PCYT2,MAD2L2,GLRX5,OGDH,UBE2J2,CSK",LV1,2
gene_set_increase_2_and_3,0.2517483,0.2517483,0.07871138,0.5391549,1.078565,63,"CHERP,RANGAP1,HNRNPL,RPS2,E4F1,TAF1C,GATAD2A,SAFB,TAF6,FBL,LSM4,SUPT5H,CHD4,PFDN6,SLC35B2,USP39,POLR3E,POLR2C,RPLP0,SREBF2,ACACA,RPL18",LV1,2
gene_set_decrease_-2_and_-3,0.1628372,0.2157842,0.10357633,0.5784179,1.137180,35,"PTBP1,KEAP1,PEX14,DLST,PCYT2,MAD2L2,GLRX5,OGDH,UBE2J2,CSK",LV1,3
gene_set_increase_2_and_3,0.2157842,0.2157842,0.08705159,0.5391549,1.087494,63,"CHERP,RANGAP1,HNRNPL,RPS2,E4F1,TAF1C,GATAD2A,SAFB,TAF6,FBL,LSM4,SUPT5H,CHD4,PFDN6,SLC35B2,USP39,POLR3E,POLR2C,RPLP0,SREBF2,ACACA,RPL18",LV1,3


In [15]:
write_tsv(df, "/home/miltondp/projects/labs/greenelab/phenoplier/base/data/crispr_screen/fsgea-all_lvs.tsv")

# Quick analyses

In [36]:
df %>% filter(lv == "LV100" & pathway == "gene_set_increase_2_and_3") %>% arrange(desc(padj))

pathway,pval,padj,log2err,ES,NES,size,leadingEdge,lv,rep_idx
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<int>
gene_set_increase_2_and_3,0.10489510,0.10489510,0.1335550,0.7069861,1.480363,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,3
gene_set_increase_2_and_3,0.10289710,0.10289710,0.1350020,0.7069861,1.478466,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,5
gene_set_increase_2_and_3,0.10289710,0.10289710,0.1350020,0.7069861,1.481544,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,7
gene_set_increase_2_and_3,0.10189810,0.10189810,0.1357409,0.7069861,1.461778,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,6
gene_set_increase_2_and_3,0.10189810,0.10189810,0.1357409,0.7069861,1.477454,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,9
gene_set_increase_2_and_3,0.09790210,0.09790210,0.1388051,0.7069861,1.494823,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,8
gene_set_increase_2_and_3,0.09490509,0.09490509,0.1412251,0.7069861,1.504016,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,4
gene_set_increase_2_and_3,0.08991009,0.08991009,0.1455161,0.7069861,1.482328,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,1
gene_set_increase_2_and_3,0.08891109,0.08891109,0.1464162,0.7069861,1.484432,40,"PCBP1,IGF1R,UXT,SNRPD1,GTF2H1",LV100,2


In [44]:
df_signif <- df %>% group_by(lv, pathway) %>% summarize(max_padj = max(padj)) %>% filter(max_padj < 0.05)

`summarise()` has grouped output by 'lv'. You can override using the `.groups` argument.



In [52]:
df_signif %>% arrange(max_padj)

lv,pathway,max_padj
<chr>,<chr>,<dbl>
LV707,gene_set_increase_2_and_3,3.477712e-07
LV678,gene_set_decrease_-2_and_-3,5.210582e-07
LV905,gene_set_increase_2_and_3,8.587291e-05
LV915,gene_set_increase_2_and_3,1.274721e-04
LV750,gene_set_increase_2_and_3,1.126546e-03
LV341,gene_set_increase_2_and_3,1.566165e-03
LV64,gene_set_decrease_-2_and_-3,2.411481e-03
LV897,gene_set_decrease_-2_and_-3,3.221430e-03
LV575,gene_set_decrease_-2_and_-3,8.551656e-03
